In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import qiskit
from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import QSVM
from qiskit.aqua.components.multiclass_extensions import one_against_rest, all_pairs
from qiskit.aqua.components.feature_maps import SecondOrderExpansion
from qiskit.aqua.input import ClassificationInput
from qiskit.aqua import run_algorithm
backend = BasicAer.get_backend('qasm_simulator')
#For any classification problem you will want to set this to metrics=['accuracy'].

In [ ]:
from qiskit import IBMQ
token = 'ADD_TOKEN_HERE'
#IBMQ.enable_account(token)

In [ ]:

#IBMQ.save_account(token,overwrite=True)
provider = IBMQ.load_account()
backend = IBMQ.backends(name='ibmqx5')


In [ ]:
IBMQ.update_account()

In [ ]:
my_provider = IBMQ.get_provider()
backend = my_provider.get_backend('ibmq_qasm_simulator')
#backend = IBMQ.get_backend(name, hub='HUB') #ibmqx4 #ibmq_16_melbourne

In [ ]:
#IBMQ.load_account()
#my_provider = IBMQ.get_provider()
#backend = my_provider
IBMQ.providers()


In [ ]:
import qiskit
backends = []
def getBack():
    for provider in IBMQ.providers():

        for backend in provider.backends():

            backends.append(backend)

    unique_hardware_backends = []

    unique_names = []

    for back in backends:

        if back.name() not in unique_names and not back.configuration().simulator:

            unique_hardware_backends.append(back)

            unique_names.append(back.name())

    if not unique_hardware_backends:

        print("Not available")

    return unique_hardware_backends
getBack()

In [ ]:
data_1_Bid = pd.read_csv("USDJPY_Long_Bid.csv")
data_1_Ask = pd.read_csv("USDJPY_Long_Ask.csv")
data = pd.DataFrame({"AskClose":data_1_Ask['Close'], 
                         "BidClose":data_1_Bid['Close']}) 

In [ ]:
#Used to prep data for classifiers, returns dataframe then array that contains lags
def Prep(data, lag_num):
    data = data.dropna()
    df = pd.DataFrame(data[['AskClose', 'BidClose']].mean(axis=1), columns=['midclose'])
    #print(data[:5]/data[:5].shift(1))
    df['returns'] = np.log(df / df.shift(1))
    lags = lag_num
    cols = []
    for lag in range(1, lags + 1):
        col = 'lag_%s' % lag
        df[col] = df['returns'].shift(lag)
        cols.append(col)
    
    df = df.dropna()
    return df, cols;

In [ ]:
df_1, cols = Prep(data,4)
print(df_1[cols])

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(df_1[cols][83100:].to_numpy(), df_1['returns'][83100:].to_numpy(), test_size=0.2)


train_x = np.sign(train_x)
test_x = np.sign(test_x)
train_y = np.sign(train_y)
test_y = np.sign(test_y)


num_features = 4
training_size = 1660
test_size = 66640
feature_map = SecondOrderExpansion(feature_dimension=num_features, depth=2)

In [ ]:
params = {
            'problem': {'name': 'classification'},
            'algorithm': {
                'name': 'QSVM',
            },
            'multiclass_extension': {'name': 'OneAgainstRest'},
            'feature_map': {'name': 'SecondOrderExpansion', 'depth': 2 }
}
 

training_dataset={'Sell':train_x[train_y==-1],
                'Nothing':train_x[train_y==0],
                'Buy':train_x[train_y==1]}
test_dataset={'Sell':test_x[test_y==-1],
                        'Nothing':test_x[test_y==0],
                        'Buy':test_x[test_y==1]}
total_arr = np.concatenate((test_dataset['Sell'],test_dataset['Nothing'],test_dataset['Buy']))
alg_input = ClassificationInput(training_dataset, test_dataset, total_arr)


In [ ]:
%time result = run_algorithm(params, algo_input=alg_input, backend=backend)


In [ ]:
q_instance = QuantumInstace(skip_qobj_validation=False)
params = {
            'problem': {'name': 'classification', 'random_seed': 794},
            'algorithm': {
                'name': 'QSVM',
            },
            'backend': {'shots': 1024},
            'multiclass_extension': {'name': 'OneAgainstRest'},
            #'feature_map': {'name': 'SecondOrderExpansion', 'depth': 2, 'entangler_map': [[0, 1]]}
            'feature_map': {'name': 'SecondOrderExpansion', 'depth': 2}
     
        }

In [ ]:
total_arr.shape

In [ ]:
train_x.shape[1]

In [ ]:
test_y.shape[1]

In [ ]:
total_arr

In [ ]:
test_dataset['B']

In [ ]:
train_x[train_y]

In [ ]:
train_y==-1

In [ ]:
from qiskit import Aer
for backend in Aer.backends():
    print(backend.name())